In [12]:
import os
os.environ["SIMCORE_CONFIG_PATH"] = r"C:\Users\anderegg\Desktop\alternative_config.json"

In [13]:
import simcoreapi
from simcoreapi import PORTS



In [14]:
import plotly.offline as offline
import plotly.figure_factory as ff
import plotly.graph_objs as go
import pandas as pd
import os

offline.init_notebook_mode(connected=True)

column_name_map = {0: {"name":"time [s]"}, 1: {"name":"rate"}, 2: {"name":"demand"}, 
                   3: {"name":"move/(max - min) sympathetic efferent", "scale":1/0.0008}, 
                   4: {"name":"threshold indirect parasympathetic efferent", "scale":-1.0},
                   5: {"name":"direct parasympathetic efferent", "scale":-1/0.0008}, 
                   6: {"name":"recruitment = coefficient of variation"}}
data_path = PORTS.inputs[0].value

data_frame = pd.read_csv(data_path, sep=',', names=[column_name_map[i]["name"] for i in column_name_map.keys()])

trace = go.Table(
    header=dict(
        values=data_frame.columns,
        fill=dict(color="#C2D4FF"),
        align=["left"]
    ),
    cells=dict(
        values=[data_frame[data_frame.columns[i]] for i in column_name_map.keys()],
        fill=dict(color="#F5F8FF"),
        align=["left"]
    )
)
data = [trace]

offline.iplot(data, filename='data-table', config={"displayModeBar": False})

In [15]:
def create_graph(data_frame, data_scaling, title, x_axis_title):
    data = [
        go.Scatter(
            x=data_frame[data_frame.columns[0]],
            y=data_frame[data_frame.columns[i]] * data_scaling[i], 
            opacity=0.5,                        
            name=data_frame.columns[i]
        ) for i in range(1,data_frame.columns.size)
    ]

    layout = go.Layout(
        title=title, 
        showlegend=True,
        xaxis=dict(
            title=x_axis_title
        ),
        yaxis=dict(
            #range=[0,.5]    
        ))
    fig = go.Figure(data=data, layout=layout)
    offline.iplot(fig, config={"displayModeBar": False})

In [16]:
title="Heart Rate(r)"
heart_axis_colums = [0,1,2,3,5]
heart_rate_data = data_frame.filter(items=[data_frame.columns[i] for i in heart_axis_colums])
heart_rate_data_scaling = [column_name_map[i]["scale"] if "scale" in column_name_map[i] else 1 for i in heart_axis_colums]
create_graph(data_frame=heart_rate_data, 
             data_scaling=heart_rate_data_scaling, 
             title=title, 
             x_axis_title="time(sec)")

In [17]:
title = "Parasympathetic Cell Activity: Cardiac Level"
NumberOfLayers=3; #number of layers
NumberOfCells=[600, 600, 600]; #cells per layer
TotalNumberOfCells = sum(NumberOfCells);
FirstCellPosition=7 #initialize where cell information begins
FirstParasympathetic=500
start_index = FirstCellPosition+0*TotalNumberOfCells+FirstParasympathetic
final_index = FirstCellPosition+0*TotalNumberOfCells+NumberOfCells[0]-1

cardiac_level_cell_activity_columns = [0, 6]
cardiac_level_cell_activity_data = data_frame.iloc[FirstParasympathetic:NumberOfCells[0]].filter(items=[data_frame.columns[i] for i in cardiac_level_cell_activity_columns])
cardiac_level_cell_activity_scaling = [column_name_map[i]["scale"] if "scale" in column_name_map[i] else 1 for i in cardiac_level_cell_activity_columns]
create_graph(data_frame=cardiac_level_cell_activity_data, 
             data_scaling=cardiac_level_cell_activity_scaling, 
             title=title, x_axis_title="time(sec)")

In [ ]:
title = "Cell Activity: Cardiac(r), Intrathoracic(g), Central(b)"
cardiac_level_cell_activity_columns = [0, 6]
cardiac_level_cell_activity_data = data_frame.filter(items=[data_frame.columns[i] for i in cardiac_level_cell_activity_columns])
cardiac_level_cell_activity_scaling = [column_name_map[i]["scale"] if "scale" in column_name_map[i] else 1 for i in cardiac_level_cell_activity_columns]
create_graph(data_frame=cardiac_level_cell_activity_data, 
             data_scaling=cardiac_level_cell_activity_scaling, 
             title=title, x_axis_title="time(sec)")

In [ ]:
import pprint
print("There are ", len(simcore._inputs), " inputs")
for _in in simcore._inputs:
    pprint.pprint(_in._asdict)

print("There are ", len(simcore._outputs), " outputs")
for _out in simcore._outputs:
    pprint.pprint(_out._asdict)
